# Projeto Final 
## Master em Jornalismo de Dados - Insper

### Tarifas aéreas no Brasil: uma análise dos dados da Anac na Ponte Aérea (Rio-SP)

Trabalho apresentado na conclusão do módulo Jornalismo de Dados e Automação do Master em Jornalismo de Dados do Insper - turma 2022/23. O projeto foi ministrado pelo professor Eduardo Vicente Gonçalves. 


### O Projeto
Um dos temas mais presentes na cobertura de um setorista de companhias aéreas é o preço dos bilhetes. As tarifas dispararam nos últimos anos por causa dos impactos da pandemia e do efeito da invasão da Rússia na Ucrânia sobre o petróleo. 

A Anac faz um monitoramento intensivo das tarifas, que é disponibilizado em um dashboard interativo bastante eficiente (https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/mercado-do-transporte-aereo), na aba Tarifas Aéreas Domésticas e Tarifas Aéreas Internacionais). Entretanto, cruzamentos mais espefícos exigem uma demanda formal via assessoria de imprensa e com prazos nem sempre favoráveis. Além disso, ao trabalhar os dados brutos se abrem novas possibilidades de visualizações. 

#### Os Dados
A Anac recolhe das companhias aéreas comerciais uma série de dados mensalmente para calcular a oferta e demanda por transporte, assim como a evolução das tarifas. No caso das tarifas, os dados representam uma fatia de cerca de 35% do total de passageiros transportados (a depender do mês). Esse recorde ocorre, conforme explica a Anac, porque os dados abrangem todas as passagens vendidas ao “público adulto em geral”, excluindo aquelas adquiridas com descontos restritos a grupos específicos, programas de milhagem, entre outras condições. 

As variáveis são diretas:
- ANO e MES: mês e ano da venda das passagens, independentemente da data do voo;
- EMPRESA: empresa que vendeu a passagem;
- ORIGEM e DESTINO;
- TARIFA: valores em reais;
- ASSENTOS: número de passagens comercializadas naquela data pela respectiva tarifa. 

Os microdados estão disponíveis no seguinte link: https://sas.anac.gov.br/sas/downloads/view/frmDownload.aspx

Para facilitar a replicabilidade/teste do projeto, os dados de janeiro de 2002 a dezembro de 2022 estão disponíveis em um arquivo zip no Google Drive (eles devem ser inseridos na pasta 'anac.nosync'): https://drive.google.com/drive/folders/1JYesuNR4Mc5QU2g4gVR9IpoqdUFrrGeq?usp=sharing

### Ferramentas

#### Spark
A base completa - janeiro de 2002 a dezembro de 2022 - tem mais de 55 milhões de registros distribuídos em 250 arquivos. Desta forma, abrir o arquivo no Pandas não seria viável na memória do computador. 

Para cruzar os dados, resolvi usar o Pyspark - interface Python para Spark e é um projeto Apache de plataforma cruzada de código aberto. 

Mas, por uma questão de capacidade de processamento, parte do tratamento dos arquivos foi feita individualmente via Pandas. Após isso, os dados são facilmente agrupados e agregados no Spark. 

#### Docker
O Spark é bastante simples de ser iniciado via Docker, método escolhido para este projeto. 
Basta clonar o repositório. Na pasta principal, digite `docker-compose up --build -d` - com isso o Docker vai construir os containeres, iniciar o sistema e manter o terminal livre (detached, ou -d). Ele vai iniciar o Spark, assim como um Worker para processar os dados. 

#### Jupyter
O Docker vai criar também um Jupyter, que pode ser acessado pelo seguinte endereço: http://localhost:8888/
Aqui é possível navegar pela pasta Anac e encontrar os notebooks necessários para o projeto.  


# Limpeza dos Dados

O primeiro desafio encontrado foi a limpeza dos dados. 
Minha máquina acabou não suportando todos os processamentos necessários no dado. Entre eles, preciso transformar variáveis, ajustar alguns CSVs com formatação e header errados. O principal desafio está na correção monetária: por padrão a Anac atualiza todos os bilhetes comercializados desde 2002 pelo IPCA acumulado no mês da última divulgação. 

In [1]:
import numpy as np
import pandas as pd
import os
#Função criada para ajustar os valores individualmente. Ela foi testada com base na Calculadora do BC. 
from monetary_correction import ipca

# Ajustando dados e trazendo tudo a valor presente
Um dos desafios mais legais foi pensar em uma função para ajustar os valores pelo IPCA.
Por se tratar de uma função que seria executada literalmente milhares de vezes, ela teria de ser rápida e bastante eficiente. 
A primeira versão corrigiu todos os dados de uma tabela de cerca de 330 mil entradas em cerca de 20 minutos. A segunda, conseguiu fazer o mesmo trabalho em cerca de 1,3 minuto - um salto bastante significativo para uma base com 250 CSVs, sendo cada um variando entre 200 mil e 500 mil entradas cada. 

In [2]:
import glob

In [12]:
# %%time
# import glob
# import os

# def update_db(date_ipca):
#     """
#     Função que pega os CSVs individualmente, limpa os dados e corrige a tarifa pelo IPCA do parâmetro month_ipca. 
#     A data precisa ser no formato "MM/YYYY"
#     """

#     if not os.path.exists("anac.nosync/real/"):
#         os.makedirs("anac.nosync/real/")
    
#     files = glob.glob("anac.nosync/*.CSV")
    
#     for file in files:
#         print(f"\r>>Processando tabela {file}", end="")
#         df = pd.read_csv(file, sep=";", encoding='latin1')
#         df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
#         df.columns = ["ANO","MES","EMPRESA","ORIGEM","DESTINO","TARIFA","ASSENTOS"]
#         df.TARIFA = df.TARIFA.str.replace(",", ".").astype(float).copy()

#         if file == files[-1]:    
#             df['REAL'] = df['TARIFA'].copy()
#         else:
#             df['REAL'] = df.apply(lambda row: ipca(row["TARIFA"], [row['MES'], row['ANO']], date_ipca)[0], axis=1)
#             # df['REAL'] = df.apply(lambda row: ipca(row["TARIFA"], f"{row['MES']}/{row['ANO']}", date_ipca)[0], axis=1)
#         df.to_csv(f"anac.nosync/real/{file.split('/')[1].split('.')[0]}.csv", index=False, sep=";")

# update_db([12,2022])

>>Processando tabela anac.nosync/202212.CSVCPU times: user 2h 32min 40s, sys: 1min 17s, total: 2h 33min 57s
Wall time: 2h 43min 3s


# Começando com o Spark
Lider com o Spark foi outro desafio.
Ele demnanda uma série de configurações completas e ajustar tudo ao meu computador foi trabalhoso. Mas consegui encontrar uma saída (com este Docker container, inspirado em um projeto do github (link nas referências). 

In [2]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
import pyspark.sql.functions as f

conf = SparkConf()
# conf.set("spark.eventLog.enabled", False)
# conf.set("spark.eventLog.dir", "/opt/spark/spark-events")
# conf.set("spark.history.fs.logDirectory", "/opt/spark/spark-events")
# conf.set("spark.executor.memory", "1024m")
# conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
# conf.set("spark.sparkContext.setLogLevel", "ERROR")

spark = SparkSession.\
        builder.\
        appName("pyspark-anac").\
        master("spark://spark-master:7077").\
        config(conf=conf).\
        getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/14 12:49:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark
# sc = spark.sparkContext
# sc.addPyFile("/opt/notebooks/udfs.py")

## Construindo o Schema
Parte importante para definir os tipos de arquivos. Uma boa definição ajuda o programa a rodar significativamente mais rápido. 

In [231]:
from pyspark.sql.types import StructType, IntegerType, StringType, FloatType, StructField, DoubleType, ArrayType

schema = StructType(
    [
        StructField('ANO', IntegerType(), True), 
        StructField('MES', IntegerType(), True), 
        StructField('EMPRESA', StringType(), True), 
        StructField('ORIGEM', StringType(), True), 
        StructField('DESTINO', StringType(), True), 
        StructField('TARIFA', FloatType(), True), 
        StructField('ASSENTOS', IntegerType(), True),
        StructField('REAL', FloatType(), True),
    ]
)

Agora vamos ler todos os arquivos editados de uma só vez. Isso é possível com o "*" antes do ".csv", que significa que o Spark irá ler todos os arquivos CSVs no diretório. Importante: todos precisam ter a mesma quantidade de colunas.

In [5]:
sdf = spark.read.csv('anac.nosync/real/*.csv', header=True, sep=";", schema=schema)

In [5]:
#Vamos contar a quantidade de linhas
#sdf.count()

In [6]:
#Uma breve análise exploratória dos dados
#sdf.summary().show()

In [6]:
#Vendo apenas as primeiras três linhas da tabela. 
sdf.limit(3).show()

+----+---+-------+------+-------+------+--------+---------+
| ANO|MES|EMPRESA|ORIGEM|DESTINO|TARIFA|ASSENTOS|     REAL|
+----+---+-------+------+-------+------+--------+---------+
|2021| 10|    AZU|  SBAC|   SBAR| 565.9|       1|616.34424|
|2021| 10|    AZU|  SBAC|   SBBR| 409.9|       1|446.43842|
|2021| 10|    AZU|  SBAC|   SBBV| 748.9|       4| 815.6568|
+----+---+-------+------+-------+------+--------+---------+



# Ajustar todos os valores pela inflação de Dezembro de 2022

Nao funciona aqui por questão de processamento. Por isso, exacutamos via pandas e editando e corrigindo os arquivos individualmente. 

```
def valor_real(value, date_start):
    return ipca(value, date_start, "12/2022")[0]

vreal = f.udf(valor_real, DoubleType())    # registering as udf here


sdf = sdf.withColumn(
    'REAL',
    vreal(
        f.col("TARIFA"), 
        f.concat(f.col("MES"), f.lit("/"), f.col("ANO")),
    )
)

sdf = sdf.select("ANO", "MES", "ASSENTOS", "REAL")
````


# Dado final

## Agrupamento por ano e mês
Primeiro vamos agrupar os dados pelas informações que a gente quer. No caso vamos pegar as tarifas médias por mês. 

In [7]:
data_by_date = sdf.groupby("ANO", "MES").agg(
    f.sum("ASSENTOS").alias("TOTAL"),
    f.collect_list(f.col('REAL')),
    f.collect_list(f.col('ASSENTOS')),
)

In [8]:
data_by_date.columns

['ANO',
 'MES',
 'TOTAL',
 'collect_list(REAL)',
 'collect_list(TARIFA)',
 'collect_list(ASSENTOS)']

In [9]:
#precisamos de uma média ponderada para calcular a real representatividade das tarifas considerando o número de assentos. 
def get_average(values, weights):
    return float(np.average(values, weights=weights)) 
waverage = f.udf(get_average, DoubleType())    # registering as udf here

data_by_date = data_by_date.withColumn(
    'tarifa_media_real',
    waverage(f.col("collect_list(REAL)"), f.col("collect_list(ASSENTOS)"))
)

In [10]:
data_by_date = data_by_date.select("ANO", "MES", "TOTAL", "tarifa_media_real")

In [11]:
data_by_date.cache() #salvando a base no cache para usar depois. 

DataFrame[ANO: int, MES: int, TOTAL: bigint, tarifa_media: double, tarifa_media_real: double]

In [12]:
data_by_date.limit(3).show()

+----+---+-------+-----------------+-----------------+
| ANO|MES|  TOTAL|     tarifa_media|tarifa_media_real|
+----+---+-------+-----------------+-----------------+
|2022| 10|2450901| 638.356869621899|648.7533615424431|
|2012| 10|4443441|309.6230932886985|567.5237020907284|
|2007|  6|1366528|280.3493941729688|681.8395058799156|
+----+---+-------+-----------------+-----------------+



In [25]:
# #Jogando para CSV para futura consulta sem precisar rodar de novo
# data_by_date.select("ANO", "MES", "TOTAL", "Weighted_average")\
#     .repartition(1)\
#     .write.format("com.databricks.spark.csv")\
#     .option("header", "true")\
#     .save("tarifa_real.csv")

## Análise Tarifas

### Mensais
- Criar gráficos mostrando a evolução das tarifas

In [14]:
import altair as alt

In [15]:
df_final = data_by_date.toPandas()
df_final["DATA"] = df_final["MES"].astype(str) + "/" + df_final["ANO"].astype(str)
df_final.DATA = df_final.DATA.apply(pd.to_datetime,format="%m/%Y")

In [16]:
df_final.head(2)

,ANO,MES,TOTAL,tarifa_media,tarifa_media_real,DATA
0,2022,10,2450901,638.356870,648.753362,2022-10-01
1,2012,10,4443441,309.623093,567.523702,2012-10-01


In [17]:
#Pegando os dados de oferta e demanda para dar ideia da queda do setor contra a tarifa. 
oferta_e_demanda = pd.read_csv("https://www.gov.br/anac/pt-br/assuntos/regulados/empresas-aereas/links-de-informacoes/DemandaeOferta.csv", sep=";")
oferta_e_demanda_domestica = oferta_e_demanda[oferta_e_demanda["natureza"] == "DOMÉSTICA"]
oferta_group = oferta_e_demanda_domestica.groupby(["ano", "mes"]).agg(passageiros_pg=("passageiros_pg", "sum")).reset_index()
oferta_group.columns = ["ANO", "MES", "PASSAGEIROS_PG"]
oferta_group["DATA"] = pd.to_datetime(dict(year=oferta_group.ANO, month=oferta_group.MES, day=1)) #criando uma nova coluna
oferta_group.head(2)

,ANO,MES,PASSAGEIROS_PG,DATA
0,2000,1,2572833.0,2000-01-01
1,2000,2,2212693.0,2000-02-01


In [18]:
df_final = pd.merge(df_final, oferta_group[["ANO", "MES", "PASSAGEIROS_PG"]], on=["ANO", "MES"])

In [19]:
base = alt.Chart(df_final[df_final["DATA"].dt.year >= 2010]).encode(x=alt.X("yearmonth(DATA):T", axis=alt.Axis(labelAngle=325)))

bar = base.mark_bar().encode(
    alt.X('yearmonth(DATA):T', title="Data"),
    alt.Y("tarifa_media_real", title="Tarifa média"),
    tooltip = ["DATA", "tarifa_media_real"],
)
line =  base.mark_line(
    color="red",
    opacity=0.3,
).encode(y=alt.Y('PASSAGEIROS_PG:Q', axis=alt.Axis(grid=True)))

(bar + line).resolve_scale(y='independent').properties(width=600)

alt.LayerChart(...)

### Tarifas Anuais

In [42]:
df_by_year = df_final.groupby("ANO").agg(
    tarifa_media_ano=("tarifa_media_real", "mean")
).reset_index()
#Fazendo a mesma coisa com os dados de oferta para criar o mesmo gráfico
oferta_group_by_year = oferta_group.groupby("ANO").agg(passageiros=("PASSAGEIROS_PG", "sum")).reset_index()
df_final_by_year = pd.merge(df_by_year, oferta_group_by_year, on=["ANO"])

df_final_by_year.head(2)

#### 2022 contra 2021

In [58]:
atual = float(df_final_by_year[df_final_by_year["ANO"] == 2022]["tarifa_media_ano"])
anterior = float(df_final_by_year[df_final_by_year["ANO"] == 2021]["tarifa_media_ano"])
anterior_2019 = float(df_final_by_year[df_final_by_year["ANO"] == 2019]["tarifa_media_ano"])

print(f"A tarifa média em 2022 foi {round(((atual - anterior)/anterior) *100, 2)}% mais cara do que 2021 e {round(((atual - anterior)/anterior_2019) *100, 2)}% acima de 2019")

A tarifa média em 2022 foi 26.558229950324563% mais cara do que 2021 e 25.932402758541095% acima de 2019


In [52]:
base = alt.Chart(df_final_by_year[df_final_by_year["ANO"] >= 2010], title="Tarifa Média Anual").encode(x=alt.X("ANO:N", axis=alt.Axis(labelAngle=325)))

bar = base.mark_bar().encode(
    alt.X('ANO:N', title="Ano"),
    alt.Y("tarifa_media_ano", title="Tarifa média"),
    tooltip = ["ANO", "tarifa_media_ano"],
)
line =  base.mark_line(
    color="red",
    opacity=0.3,
).encode(y=alt.Y('passageiros:Q', axis=alt.Axis(grid=True)))

(bar + line).resolve_scale(y='independent').properties(width=600)

alt.LayerChart(...)

## Tarifa por companhia aérea
- Criar gráfico por companhia aérea

Um gráfico interativo com a variação da tarifas por companhia aérea. Ver o que tá pegando com elas. 


In [76]:
data = sdf.groupby("ANO", "MES", "EMPRESA").agg(
    f.sum("ASSENTOS").alias("TOTAL"),
    f.collect_list(f.col('REAL')),
    f.collect_list(f.col('ASSENTOS')),
)

#precisamos de uma média ponderada para calcular a real representatividade das tarifas considerando o número de assentos. 
def get_average(values, weights):
    return float(np.average(values, weights=weights)) 

waverage = f.udf(get_average, DoubleType())    # registering as udf here

data = data.withColumn(
    'tarifa_real',
    waverage(
        f.col("collect_list(REAL)"), f.col("collect_list(ASSENTOS)")
    )
)

In [79]:
data = data.select("ANO", "MES", "EMPRESA", "tarifa_real")

In [80]:
data.cache()

DataFrame[ANO: int, MES: int, EMPRESA: string, tarifa_real: double]

In [81]:
data.limit(3).show()

+----+---+-------+------------------+
| ANO|MES|EMPRESA|       tarifa_real|
+----+---+-------+------------------+
|2020| 11|    GLO|417.92073660891043|
|2019|  5|    PTB| 590.2504869667179|
|2017|  4|    TAM|373.71520858248743|
+----+---+-------+------------------+



In [109]:
data_by_cia = data.toPandas()

In [199]:
cias = ["AZU", "GLO", "TAM"] 

In [200]:
data_by_cia = data_by_cia[data_by_cia["EMPRESA"].isin(cias)].copy()

In [201]:
data_by_cia["DATA"] = data_by_cia["MES"].astype(str) + "/" + data_by_cia["ANO"].astype(str)
data_by_cia.DATA = data_by_cia.DATA.apply(pd.to_datetime,format="%m/%Y")
data_by_cia = data_by_cia[data_by_cia["DATA"].dt.year >= 2013]

In [208]:
base = alt.Chart(data_by_cia).encode(x='yearmonth(DATA):T')

columns = sorted(data_by_cia.EMPRESA.unique())
selection = alt.selection_single(
    fields=['DATA'], nearest=True, on='mouseover', empty='none', clear='mouseout'
)
lines = base.mark_line().encode(y='tarifa_real:Q', color='EMPRESA:N')
points = lines.mark_point().transform_filter(selection)

rule = base.transform_pivot(
    'EMPRESA', value='tarifa_real', groupby=['DATA']
).mark_rule().encode(
    opacity=alt.condition(selection, alt.value(0.3), alt.value(0)),
    tooltip=[alt.Tooltip(c, type='quantitative') for c in columns]
).add_selection(selection)

lines + points + rule

alt.LayerChart(...)

## Tarifa por faixa de preço
Vamos separar as tarifas por um intervalo de 100 reais. O mesmo gráfico pode ser usado nos dados locais.

In [266]:
tiket_range = sdf.groupby("ANO", "MES", "REAL").agg(
    f.sum("ASSENTOS").alias("TOTAL"),
    f.collect_list(f.col('ASSENTOS')),
)

In [265]:
tiket_range.cache()

DataFrame[ANO: int, MES: int, REAL: float, TOTAL: bigint]

In [ ]:
#precisamos de uma média ponderada para calcular a real representatividade das tarifas considerando o número de assentos. 
def percentage(values, weights):
    return  

waverage = f.udf(get_average, DoubleType())    # registering as udf here

data = data.withColumn(
    'tarifa_real',
    waverage(
        f.col("collect_list(REAL)"), f.col("collect_list(ASSENTOS)")
    )
)

## Tarifa por Rota - Foco Ponte Aérea Rio SP 

Congonhas: SBSP

Santos Dumont: SBRJ

In [214]:
ponte_aerea = sdf\
    .filter(
        (
            (
                f.col('ORIGEM') == "SBSP") & (f.col('DESTINO') == "SBRJ"
            ) | (
                f.col('ORIGEM') == "SBRJ") & (f.col('DESTINO') == "SBSP"
            )
        )
    )

ponte_aerea.cache()
ponte_aerea.limit(3).show()

+----+---+-------+------+-------+-------+--------+---------+
| ANO|MES|EMPRESA|ORIGEM|DESTINO| TARIFA|ASSENTOS|     REAL|
+----+---+-------+------+-------+-------+--------+---------+
|2021| 10|    AZU|  SBRJ|   SBSP|1008.27|       3| 1098.147|
|2021| 10|    AZU|  SBRJ|   SBSP|1027.89|       3|1119.5159|
|2021| 10|    AZU|  SBRJ|   SBSP| 1027.9|      19|1119.5269|
+----+---+-------+------+-------+-------+--------+---------+



In [218]:
ponte_aerea.limit(2).show()

+----+---+-------+------+-------+-------+--------+---------+
| ANO|MES|EMPRESA|ORIGEM|DESTINO| TARIFA|ASSENTOS|     REAL|
+----+---+-------+------+-------+-------+--------+---------+
|2021| 10|    AZU|  SBRJ|   SBSP|1008.27|       3| 1098.147|
|2021| 10|    AZU|  SBRJ|   SBSP|1027.89|       3|1119.5159|
+----+---+-------+------+-------+-------+--------+---------+



### Gráfico apenas com valores das tarifas no passar do tempo

In [219]:
ponte_aerea_group = ponte_aerea.groupby("ANO", "MES").agg(
    f.sum("ASSENTOS").alias("TOTAL"),
    f.collect_list(f.col('REAL')),
    f.collect_list(f.col('ASSENTOS')),
)

In [220]:
#precisamos de uma média ponderada para calcular a real representatividade das tarifas considerando o número de assentos. 
def get_average(values, weights):
    return float(np.average(values, weights=weights)) 
waverage = f.udf(get_average, DoubleType())    # registering as udf here

ponte_aerea_group = ponte_aerea_group.withColumn(
    'tarifa_media_real',
    waverage(f.col("collect_list(REAL)"), f.col("collect_list(ASSENTOS)"))
)

In [221]:
ponte_aerea_group.limit(3).show()

+----+---+------+--------------------+----------------------+------------------+
| ANO|MES| TOTAL|  collect_list(REAL)|collect_list(ASSENTOS)| tarifa_media_real|
+----+---+------+--------------------+----------------------+------------------+
|2021| 10|118393|[1098.147, 1119.5...|  [3, 3, 19, 15, 4,...| 465.9539508494635|
|2021| 11|144218|[88.09931, 95.704...|  [31, 1, 1, 664, 3...|454.52439056113144|
|2015| 11|167082|[162.23549, 152.6...|  [2, 3, 2, 30, 2, ...|401.08876547571003|
+----+---+------+--------------------+----------------------+------------------+



### Gráfico com divisão das tarifas no trecho por valores.

# Referências
- Docker Pyspark: https://github.com/mrn-aglic/spark-standalone-cluster
- Calculo da Variação pelo IPCA: https://estudaradm.com.br/macroeconomia/calcular-inflacao-acumulada-em-12-meses-exemplos-e-exercicios/?unapproved=1687&moderation-hash=42b2b86a1784537b7b5a3c2fa55d719e#comment-1687 (OBS: obrigado especial ao criador desta publicação. Todos os blogs que eu li falando que iriam ensinar a deflacionar um número apenas usavam a calculadora do BC)

# Dezembro 22 - Dezembro 2019

In [58]:
dezembro = pd.read_csv("anac.nosync/202212.CSV", sep=";", encoding="latin1")
# dezembro.columns = ['ANO', 'MES', 'EMPRESA', 'ORIGEM', 'DESTINO', 'TARIFA', 'ASSENTOS']
dezembro.TARIFA = dezembro.TARIFA.str.replace(",", ".").astype(float)


In [67]:
# Aqui com os dados base julho de 2022
wm = lambda x: np.average(x, weights=dezembro.loc[x.index, "ASSENTOS"])

media_dezembro = dezembro.groupby(['ANO', 'MES']).agg(price_weighted_mean=("TARIFA", wm))
media_dezembro

,,price_weighted_mean
ANO,MES,
2022,12,650.817921
